In [125]:
import bs4 
import urllib
from urllib import request
import pandas as pd
import twitter
import json
import time
import os
import ast
import datetime

def extract_influencers(file):
    request_text = open(file,'r')
    page = bs4.BeautifulSoup(request_text,'lxml')
    return [x.get_text().split("https://twitter.com/")[1] for x in page1.findAll("a") if "twitter.com" in x.get_text()]


In [108]:
files = ["data tweeter/topinfluencers1.htm", "data tweeter/topinfluencers2.html"]
influencers = pd.DataFrame()

for item in files:
    temp = extract_influencers(item)
    print(temp)
    influencers = influencers.append(temp, ignore_index = True)

# remove duplicates from various lists
dedup = influencers[influencers.duplicated(keep = "first") == False]

# dump to CSV
dedup.to_csv("data tweeter/influencers.csv")

['VitalikButerin', 'SatoshiLite', 'officialmcafee', 'aantonop', 'rogerkver', 'naval', 'ErikVoorhees', 'NickSzabo4', 'justinsuntron', 'brian_armstrong', 'fredwilson', 'cz_binance', 'cdixon', 'TuurDemeester', 'WhalePanda', 'lopp', 'barrysilbert', 'adam3us', 'APompliano', 'VentureCoinist', 'ToneVays', 'PeterLBrandt', 'VinnyLingham', 'AriDavidPaul', 'businessinsider/status/943979396758618113', 'CharlieShrem', 'peterktodd', 'jimmysong', 'CryptoYoda1338', 'gavinandresen', 'CryptoCobain', 'crypto_rand', 'notsofast', 'CarpeNoctom', 'CryptoBull', 'cburniske', 'balajis', 'JihanWu', 'maxkeiser', 'AngeloBTC', 'Excellion', 'iamjosephyoung', 'twobitidiot', 'CryptoHustle', 'ethereumJoseph', 'FEhrsam', 'woonomic', 'zooko', 'Melt_Dem', 'laurashin', 'CryptOrca', 'novogratz', 'TheCryptoDog', 'fluffypony', 'arrington', 'CryptoCred', 'cryptomanran', 'bitcoin_dad', 'ThisIsNuse', 'el33th4xor', 'alistairmilne', 'cryptomocho', 'TimDraper', 'cryptoSqueeze', 'jgarzik', 'CryptoDonAlt', 'tylerwinklevoss', 'cryptop

In [98]:
api = twitter.Api(consumer_key='xxx',
                  consumer_secret='xxx',
                  access_token_key='xxx',
                  access_token_secret='xxx')

In [126]:
infl = pd.read_csv("data tweeter/influencers.csv", index_col = 0)
#remove 'business insider xxx' infl.drop(24)

In [127]:
empty = pd.DataFrame()
tweet_depth = 20 # nb of cycle per influencer, with each cycle collecting 200 tweets 

# collect all tweets of our stars contributors
for names in infl.loc[:,'0']:
    i = 0
    tweets = empty
    last = None
    while i < tweet_depth:
        statuses = api.GetUserTimeline(screen_name = str(names), count = 200, max_id = last, exclude_replies = True)
        for items in statuses:
            line = {"ID":int(items.id), "ScreenName": str(names),"Created": items.created_at, "text":items.text}
            tweets = tweets.append(line, ignore_index = True)
        #        tweets = tweets.append(statuses)
        last = tweets["ID"].min()
        i+=1
    time.sleep(1)
    tweets.to_csv("data tweeter/tweets/{}_{}.csv".format(names,last))
    

TwitterError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [129]:
# recompile these tweets into 1 nice file
files = os.listdir("data twitter/tweets/")
files.sort()
data = pd.DataFrame()

for items in files: 
    temp = pd.read_csv('data twitter/tweets/' + items)
    data = data.append(temp, ignore_index = True)

tweets = tweets[tweets.duplicated() == False]   
data.to_csv("data twitter/tweets.csv")

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [130]:
data = pd.read_csv("data twitter/tweets.csv", index_col = 0)

In [133]:
data.sort_values("Created")

,Created,ID,ScreenName,Unnamed: 0.1,text
36262,Fri Apr 01 01:13:28 +0000 2016,7.157086e+17,cdixon,629,RT @nitashatiku: Meet the breakfast club of st...
36261,Fri Apr 01 05:55:41 +0000 2016,7.157797e+17,cdixon,628,RT @benthompson: Read this tweetstorm by @conn...
864,Fri Apr 01 06:40:49 +0000 2016,7.157910e+17,AngeloBTC,321,https://t.co/hkaTAbrBEu\n\n#Savage https://t.c...
36260,Fri Apr 01 07:27:22 +0000 2016,7.158027e+17,cdixon,627,40 Years of Apple in Pictures https://t.co/Srm...
57359,Fri Apr 01 14:27:05 +0000 2016,7.159083e+17,rogerkver,994,RT @BTCTN: Trump: ‘I’ll Build a Wall Around Ba...
43425,Fri Apr 01 20:52:05 +0000 2011,5.392260e+16,gavinandresen,937,RT @EconTalker: Coming Monday to EconTalk--Gav...
57912,Fri Apr 03 14:35:04 +0000 2015,5.840012e+17,rogerkver,1547,"""Bitcoin's success is a strong, undeniable sig..."
57911,Fri Apr 03 14:38:24 +0000 2015,5.840020e+17,rogerkver,1546,"RT @maxkeiser: On this Easter 2015, heed the p..."
57910,Fri Apr 03 15:17:35 +0000 2015,5.840119e+17,rogerkver,1545,"""The Blockchain was designed specifically to b..."
57909,Fri Apr 03 15:27:18 +0000 2015,5.840143e+17,rogerkver,1544,"Bitcoin is ""one of the most profound beneficia..."
